# Data Exploration and Analysis

The two training data ESRI shapefiles provided `Yearly Training Sites_kenya and Uganda\\Yearly_Point_Data_Uganda.shp` and `Yearly Training Sites_kenya and Uganda\\Yearly_Point_Data_Kenya.shp` did not have any coordinate reference system. 
They were assigned the `EPSG:4326` coordinate reference system in QGIS before being merged and exported as a `GeoJSON` file. The two columns `layer` and `path` were dropped from the file in QGIS and the resulting `GeoJSoN` file was uploaded into the sandbox.

## Load packages

In [1]:
import json

import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box

## Load the training data

In [2]:
training_data_fp = "data/kenya_uganda_yearly_point_training_sites.geojson"

In [3]:
training_data_gdf = gpd.read_file(training_data_fp)
training_data_gdf.head()

,plotid,year,country,country_id,latitude,longitude,landcover,geometry
0,2,1985,kenya,7,4.371003,35.746118,Open Grassland,POINT (35.74612 4.37100)
1,3,1985,kenya,7,-0.524994,40.402437,Wooded Grassland,POINT (40.40244 -0.52499)
2,4,1985,kenya,7,0.535051,40.435959,Wooded Grassland,POINT (40.43596 0.53505)
3,7,1985,kenya,7,0.251115,36.293126,Cropland,POINT (36.29313 0.25112)
4,8,1985,kenya,7,4.099182,37.396270,Open Grassland,POINT (37.39627 4.09918)


In [4]:
# Check if the training data has a coordinate reference system.
training_data_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Data Cleaning

The training data is in point geometry. 
The shapefiles each have 8 columns:
* `plotid`: 
* `year`: The year the training data was collected.
* `country`: The country either Kenya or Uganda.
* `country_id`: The country id is either `5` for `uganda` or `7` for `kenya`.
* `lattitude`: The lattitude coordinate (EPSG:4326) for the training data point.
* `longitude`: The longitude coordinate (EPSG:4326) for the training data point.
* `landcover`: The land cover class the point represents.
* `geometry`: The point geometry of training point in EPSG:4326 crs. 


### Assign a unique integer value to represent each land cover class

In [5]:
# Get a list of the unique land cover classes in the data.
class_names = np.unique(training_data_gdf.landcover)
class_names

array(['Cropland', 'Dense Forest', 'Open Forest', 'Open Grassland',
       'Open Water', 'Otherland', 'Settlements', 'Vegetated Wetland',
       'Wooded Grassland'], dtype=object)

In [6]:
# Get a list of unique integer values to assign to each land cover class.
class_values = list(range(1, len(class_names) + 1))
class_values

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
# Generate a dictionary mapping the 'string' name of the classes to the integer values that represent the land cover classes in the training data.
class_dict = dict(zip(class_names, class_values))
class_dict

{'Cropland': 1,
 'Dense Forest': 2,
 'Open Forest': 3,
 'Open Grassland': 4,
 'Open Water': 5,
 'Otherland': 6,
 'Settlements': 7,
 'Vegetated Wetland': 8,
 'Wooded Grassland': 9}

In [8]:
# Class colors
class_colors = [
    "#F096FF",
    "#007800",
    "#648C00",
    "#FFBB22",
    "#0032C8",
    "#B4B4B4",
    "#FA0000",
    "#0096A0",
    "#FFFF4C",
]

class_color_dict = dict(zip(class_names, class_colors))
class_color_dict

{'Cropland': '#F096FF',
 'Dense Forest': '#007800',
 'Open Forest': '#648C00',
 'Open Grassland': '#FFBB22',
 'Open Water': '#0032C8',
 'Otherland': '#B4B4B4',
 'Settlements': '#FA0000',
 'Vegetated Wetland': '#0096A0',
 'Wooded Grassland': '#FFFF4C'}

In [9]:
labels = {}
for class_name, class_value in class_dict.items():
    labels[class_value] = {'color': class_color_dict[class_name], 'flag': class_name}
    
labels

{1: {'color': '#F096FF', 'flag': 'Cropland'},
 2: {'color': '#007800', 'flag': 'Dense Forest'},
 3: {'color': '#648C00', 'flag': 'Open Forest'},
 4: {'color': '#FFBB22', 'flag': 'Open Grassland'},
 5: {'color': '#0032C8', 'flag': 'Open Water'},
 6: {'color': '#B4B4B4', 'flag': 'Otherland'},
 7: {'color': '#FA0000', 'flag': 'Settlements'},
 8: {'color': '#0096A0', 'flag': 'Vegetated Wetland'},
 9: {'color': '#FFFF4C', 'flag': 'Wooded Grassland'}}

In [10]:
# Add a new column 'class'.
training_data_gdf["class"] = training_data_gdf["landcover"].map(class_dict)
training_data_gdf.head()

,plotid,year,country,country_id,latitude,longitude,landcover,geometry,class
0,2,1985,kenya,7,4.371003,35.746118,Open Grassland,POINT (35.74612 4.37100),4
1,3,1985,kenya,7,-0.524994,40.402437,Wooded Grassland,POINT (40.40244 -0.52499),9
2,4,1985,kenya,7,0.535051,40.435959,Wooded Grassland,POINT (40.43596 0.53505),9
3,7,1985,kenya,7,0.251115,36.293126,Cropland,POINT (36.29313 0.25112),1
4,8,1985,kenya,7,4.099182,37.396270,Open Grassland,POINT (37.39627 4.09918),4


## Convert the points to polygons.

In [11]:
# Function to get a square polygon from a point.
def polygon_from_centroid(point):
    # Buffer was assumed from the `crop_training_qgypt.geojson` file.
    buffer = 0.0015
    # Buffer fromm the number of decimal places in the coordinates.
    # buffer = 0.000001 # this does not work
    minx = point.geometry.x - buffer
    miny = point.geometry.y - buffer
    maxx = point.geometry.x + buffer
    maxy = point.geometry.y + buffer
    polygon = box(minx, miny, maxx, maxy, ccw=True)

    return polygon


# Convert the points to polygons.
training_data_gdf["polygon"] = training_data_gdf.apply(polygon_from_centroid, axis=1)

# Change the geometry of the GeoDataFrame to the `polygon` column.
training_data_gdf.set_geometry(col="polygon", drop=True, inplace=True, crs="EPSG:4326")

training_data_gdf

,plotid,year,country,country_id,latitude,longitude,landcover,geometry,class
0,2,1985,kenya,7,4.371003,35.746118,Open Grassland,"POLYGON ((35.74762 4.36950, 35.74762 4.37250, ...",4
1,3,1985,kenya,7,-0.524994,40.402437,Wooded Grassland,"POLYGON ((40.40394 -0.52649, 40.40394 -0.52349...",9
2,4,1985,kenya,7,0.535051,40.435959,Wooded Grassland,"POLYGON ((40.43746 0.53355, 40.43746 0.53655, ...",9
3,7,1985,kenya,7,0.251115,36.293126,Cropland,"POLYGON ((36.29463 0.24962, 36.29463 0.25262, ...",1
4,8,1985,kenya,7,4.099182,37.396270,Open Grassland,"POLYGON ((37.39777 4.09768, 37.39777 4.10068, ...",4
...,...,...,...,...,...,...,...,...,...
131158,1996,2017,uganda,5,-0.920187,32.356140,Open Water,"POLYGON ((32.35764 -0.92169, 32.35764 -0.91869...",5
131159,1997,2017,uganda,5,0.917003,30.356584,Open Grassland,"POLYGON ((30.35808 0.91550, 30.35808 0.91850, ...",4
131160,1998,2017,uganda,5,2.538681,32.146721,Open Grassland,"POLYGON ((32.14822 2.53718, 32.14822 2.54018, ...",4
131161,1999,2017,uganda,5,3.101875,31.890663,Open Forest,"POLYGON ((31.89216 3.10038, 31.89216 3.10338, ...",3


### Save as a GeoJSON

In [12]:
# Get the training data for each year.
years = np.unique(training_data_gdf["year"])
for year in years:
    selected_training_data = training_data_gdf[training_data_gdf["year"] == year]
    # Drop unneccesary columns.
    selected_training_data = selected_training_data[["class", "geometry"]]
    # Save to GeoJSON
    selected_training_data.to_file(
        f"data/kenya_uganda_yearly_polygon_training_sites_{year}.geojson",
        driver="GeoJSON",
    )

## Data Exploration

In [13]:
# Find the number of training points available for each land cover class for each year.
years = np.unique(training_data_gdf["year"])

print(f"Training data for all years {years.min()} to {years.max()}:")
print(training_data_gdf["landcover"].value_counts())

for year in years:
    selected_training_data = training_data_gdf[training_data_gdf["year"] == year]
    print(f"\nTraining data for the year {year}:")
    print(selected_training_data["landcover"].value_counts())

Training data for all years 1985 to 2017:
Wooded Grassland     43454
Open Grassland       30122
Cropland             23093
Open Water           11258
Otherland             9376
Open Forest           6653
Dense Forest          4929
Vegetated Wetland     1743
Settlements            535
Name: landcover, dtype: int64

Training data for the year 1985:
Wooded Grassland     1057
Open Grassland        755
Cropland              462
Open Water            308
Otherland             273
Open Forest           161
Dense Forest          119
Vegetated Wetland      39
Settlements            13
Name: landcover, dtype: int64

Training data for the year 1986:
Wooded Grassland     1377
Open Grassland        940
Cropland              606
Open Water            342
Otherland             285
Open Forest           216
Dense Forest          157
Vegetated Wetland      55
Settlements            13
Name: landcover, dtype: int64

Training data for the year 1987:
Wooded Grassland     1375
Open Grassland        941
Cro